# AML Mini-Challenge - Credit Card Affinity Modelling

Dominik Filliger & Noah Leuenberger


# Task

The task can be found [here](https://spaces.technik.fhnw.ch/storage/uploads/spaces/82/exercises/20240218__AML_Trainingscenter_MiniChallenge_Kreditkarten_Aufgabenstellung-1708412668.pdf).

# Setup

In [1]:
import pandas as pd
from datetime import datetime

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# Data Import & Preprocessing

## Accounts

In [2]:
accounts = pd.read_csv("data/account.csv", sep=";")

# Translated frequency from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
accounts['frequency'] = accounts['frequency'].map({
    "POPLATEK MESICNE": "MONTHLY_ISSUANCE",
    "POPLATEK TYDNE": "WEEKLY_ISSUANCE",
    "POPLATEK PO OBRATU": "ISSUANCE_AFTER_TRANSACTION"
})

accounts['date'] = pd.to_datetime(accounts['date'], format='%y%m%d')
accounts['frequency'] = accounts['frequency'].astype('category')

accounts.rename(columns={'date': 'account_created', 
                         'frequency': 'account_frequency'}, inplace=True)

display(accounts.head(), accounts.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   account_id         4500 non-null   int64         
 1   district_id        4500 non-null   int64         
 2   account_frequency  4500 non-null   category      
 3   account_created    4500 non-null   datetime64[ns]
dtypes: category(1), datetime64[ns](1), int64(2)
memory usage: 110.1 KB


,account_id,district_id,account_frequency,account_created
0,576,55,MONTHLY_ISSUANCE,1993-01-01
1,3818,74,MONTHLY_ISSUANCE,1993-01-01
2,704,55,MONTHLY_ISSUANCE,1993-01-01
3,2378,16,MONTHLY_ISSUANCE,1993-01-01
4,2632,24,MONTHLY_ISSUANCE,1993-01-02


None

## Clients

In [3]:
clients = pd.read_csv("data/client.csv", sep=";")

def parse_birth_number(birth_number):
    birth_number_str = str(birth_number)
    
    # Extract year, month, and day from birth number from string
    # according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
    year = int(birth_number_str[:2])
    month = int(birth_number_str[2:4])
    day = int(birth_number_str[4:6])

    # Determine sex based on month and adjust month for female clients
    # according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
    if month > 50:
        sex = "Female"
        month -= 50
    else:
        sex = "Male"

    # Validate date
    assert 1 <= month <= 12
    assert 1 <= day <= 31
    assert 0 <= year <= 99
    
    if month in [4, 6, 9, 11]:
        assert 1 <= day <= 30
    elif month == 2:
        assert 1 <= day <= 29
    else:
        assert 1 <= day <= 31

    # Assuming all dates are in the 1900s
    birth_date = datetime(1900 + year, month, day)
    return pd.Series([sex, birth_date])

clients[['client_sex', 'birth_date']] = clients['birth_number'].apply(parse_birth_number)

# Calculate 'age' assuming the reference year is 1999
clients['age'] = clients['birth_date'].apply(lambda x: 1999 - x.year)

# Drop 'birth_number' column as it is no longer needed
clients = clients.drop(columns=['birth_number'])

clients.head()

,client_id,district_id,client_sex,birth_date,age
0,1,18,Female,1970-12-13,29
1,2,1,Male,1945-02-04,54
2,3,1,Female,1940-10-09,59
3,4,5,Male,1956-12-01,43
4,5,5,Female,1960-07-03,39


## Dispositions

In [4]:
dispositions = pd.read_csv("data/disp.csv", sep=";")
dispositions['type'] = dispositions['type'].astype('category')

dispositions.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


## Orders

In [5]:
orders = pd.read_csv("data/order.csv", sep=";")

# Translated from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
orders['k_symbol'] = orders['k_symbol'].map({
    "POJISTNE": "Insurance_Payment",
    "SIPO": "Household",
    "LEASING": "Leasing",
    "UVER": "Loan_Payment"
})

orders['k_symbol'] = orders['k_symbol'].astype('category')
orders['bank_to'] = orders['bank_to'].astype('category')

orders = orders.rename(columns={'amount': 'debited_amount'})

orders.head()

,order_id,account_id,bank_to,account_to,debited_amount,k_symbol
0,29401,1,YZ,87144583,2452.0,Household
1,29402,2,ST,89597016,3372.7,Loan_Payment
2,29403,2,QR,13943797,7266.0,Household
3,29404,3,WX,83084338,1135.0,Household
4,29405,3,CD,24485939,327.0,NaN


## Transactions

In [6]:
# column 8 is the 'bank' column which contains NaNs and must be read as string
transactions = pd.read_csv("data/trans.csv", sep=";", dtype={8: str})

transactions['date'] = pd.to_datetime(transactions['date'], format='%y%m%d')

# Translated type, operations and characteristics from Czech to English
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
transactions['type'] = transactions['type'].map({
    "VYBER": "VYDAJ",
    "PRIJEM": "Credit",
    "VYDAJ": "Withdrawal"
})

transactions['operation'] = transactions['operation'].map({
    "VYBER KARTOU": "Credit Card Withdrawal",
    "VKLAD": "Credit in Cash",
    "PREVOD Z UCTU": "Collection from Another Bank",
    "VYBER": "Withdrawal in Cash",
    "PREVOD NA UCET": "Remittance to Another Bank"
})

transactions['k_symbol'] = transactions['k_symbol'].map({
    "POJISTNE": "Insurance Payment",
    "SLUZBY": "Payment on Statement",
    "UROK": "Interest Credited",
    "SANKC. UROK": "Sanction Interest",
    "SIPO": "Household",
    "DUCHOD": "Old-age Pension",
    "UVER": "Loan Payment"
})

transactions['bank'] = transactions['bank'].replace('', np.nan)

transactions['amount'] = np.where(transactions['type'] == "Credit", transactions['amount'], -transactions['amount'])
transactions.rename(columns={'type': 'transaction_type'}, inplace=True)

transactions.head()

,trans_id,account_id,date,transaction_type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,1993-01-01,Credit,Credit in Cash,700.0,700.0,NaN,NaN,NaN
1,171812,576,1993-01-01,Credit,Credit in Cash,900.0,900.0,NaN,NaN,NaN
2,207264,704,1993-01-01,Credit,Credit in Cash,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,1993-01-01,Credit,Credit in Cash,600.0,600.0,NaN,NaN,NaN
4,579373,1972,1993-01-02,Credit,Credit in Cash,400.0,400.0,NaN,NaN,NaN


## Loans

In [7]:
loans = pd.read_csv("data/loan.csv", sep=";")

loans['date'] = pd.to_datetime(loans['date'], format='%y%m%d')

loans['status'] = loans['status'].map({
    "A": "Contract finished, no problems",
    "B": "Contract finished, loan not payed",
    "C": "Contract running, OK thus-far",
    "D": "Contract running, client in debt"
})

# Rename columns
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
loans.rename(columns={
    'date': 'loan_granted_date',
    'amount': 'loan_amount',
    'duration': 'loan_duration',
    'payments': 'loan_monthly_payments',
    'status': 'loan_status'
}, inplace=True)

loans.head()

,loan_id,account_id,loan_granted_date,loan_amount,loan_duration,loan_monthly_payments,loan_status
0,5314,1787,1993-07-05,96396,12,8033.0,"Contract finished, loan not payed"
1,5316,1801,1993-07-11,165960,36,4610.0,"Contract finished, no problems"
2,6863,9188,1993-07-28,127080,60,2118.0,"Contract finished, no problems"
3,5325,1843,1993-08-03,105804,36,2939.0,"Contract finished, no problems"
4,7240,11013,1993-09-06,274740,60,4579.0,"Contract finished, no problems"


## Credit Cards

In [8]:
cards = pd.read_csv("data/card.csv", sep=";")
cards['type'] = cards['type'].astype('category')

cards['issued'] = pd.to_datetime(cards['issued'], format='%y%m%d %H:%M:%S').dt.date
cards.rename(columns={'type': 'card_type', 
                      'issued': 'card_issued'}, inplace=True)

cards.head()

,card_id,disp_id,card_type,card_issued
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05
3,70,439,classic,1994-02-08
4,577,3687,classic,1994-02-15


## Demographic data

In [9]:
districts = pd.read_csv("data/district.csv", sep=";")

# Rename columns
# according to https://sorry.vse.cz/~berka/challenge/PAST/index.html
districts.rename(columns={
    'A1': 'district_id',
    'A2': 'district_name',
    'A3': 'region',
    'A4': 'inhabitants',
    'A5': 'small_municipalities',
    'A6': 'medium_municipalities',
    'A7': 'large_municipalities',
    'A8': 'huge_municipalities',
    'A9': 'cities',
    'A10': 'ratio_urban_inhabitants',
    'A11': 'average_salary',
    'A12': 'unemployment_rate_1995',
    'A13': 'unemployment_rate_1996',
    'A14': 'entrepreneurs_per_1000_inhabitants',
    'A15': 'crimes_committed_1995',
    'A16': 'crimes_committed_1996'
}, inplace=True)

for col in ['unemployment_rate_1995', 'unemployment_rate_1996', 'crimes_committed_1995', 'crimes_committed_1996']:
    districts[col] = pd.to_numeric(districts[col], errors='coerce')

districts = districts.astype({'region': 'category', 
                              'district_name': 'category'})

districts.head()

,district_id,district_name,region,inhabitants,small_municipalities,medium_municipalities,large_municipalities,huge_municipalities,cities,ratio_urban_inhabitants,average_salary,unemployment_rate_1995,unemployment_rate_1996,entrepreneurs_per_1000_inhabitants,crimes_committed_1995,crimes_committed_1996
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677.0,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159.0,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824.0,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244.0,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616.0,3040


# EDA

Following the documentation of the dataset, there are multiple relationships which need to be validated. https://sorry.vse.cz/~berka/challenge/PAST/index.html

## Relationships

The ERD according to the descriptions on https://sorry.vse.cz/~berka/challenge/PAST/index.html

[![](https://mermaid.ink/img/pako:eNqtV1Fv4jgQ_itWXu6l7SZ0gQatTsqGdhddCxVQrXSqFJnEgLWJnbOd7bGl__3GTgA3JGyvWh7ajPPNZ898nrHz7MQ8Ic7AIWJI8Urg7JEh-AVhOHkYz9FzaeqfVIKyFcJxzAumIpqg-7_QoxOUNhoNH50jcEL1Q2zQNxo9rAZq8AQrUv45EIaCYEU5Q0MYb-BeCvJPQVi8AZ-RlAVmMUE3u8Gdw0v5L7wdXTeGE6eUWNGExmxa3YIKtY5YkS2IAOBnbbYt7S1hVwsbjmb3k9loPpqMG1YHRPl-bUCSc0lNThqzfYjlpjWWYxVvfqmi2uRamMkTI-LDgySiFsNkOryeNqyei4SI_fIn2voNq1lg9j1SHGBTEtPcxPgZxk7w1tAVve2wTDlWCGdmWkg1WVBFEhRkdWDF_D2Sm2zBU8De402mWcM1FjhWRNCfZt_WkjSfBuNZELYIrQRmcp-qubaAq1XptyXMriqbsmXXVjJD3SVUffhG1ToR-AmnDVCeE1EW52vmO2gmJ9JqQ49TW4IXODWVDCVWPQVLSCqyfE_rYU_SpkltOxkVhYLt_YudZAFrm6hS-XYSNMkLkbG9urdg_C5ZDdcXiFjv1rqutewb7HHaKbxMir2cBjWs7GO2vNztEpB3nKl1ukFVAciGgKTCqpA71pmx6p05mA6b-jIWyaErg9F6wOR2lz3dIHc7XNPN4fkoqxTOEUjkbsbSstN66Nrz6SicN7fsfe-_P3HkVXCGM2KDxmA3wARZlfJMzUNdP8YjvowoW2PoW7iUZ8wvEF-i0WGw2SsrGIXOiFNIHJHRE9S9zRSlKuq6rtb7FRB9QjD8PkpwhJ4ceb7vH_PCy3P95n3UHbfk9hu59dtz_93kKxV5wNCQjD_NeDNrXEL2ioTGPi4tU3LaoRAL3S5eqfmgx2wx0VTDG8r9B7TmFYkkdE-h70ZBOYBmZuDYoWAky1NuajiCNZDI7-oJrWE9F0F_-N03efdavHvN6QGEILkgjBRCRnCwmBzXwr-2QQhAyGvPN88yqqAhRrGgGeho4gnNcz2IU14926tndYBaD9huz8_58_7KPDDtLsa6I4_2lVpHVxfSgSlpSROY5ADeUVVY-46oHQIpeUzNBPqU3nlVlP_PyYZpz-227Mja5SuWLSsqb3wa8wUk0bK3Ie1rTxlsDNfCZvS2PD4Hpv8y9BW2ssY5Z05GRIZpAt8ppuE-OmpNdJ_U0IQscZGaI-0FoLhQfLZhsTNQoiBnTpHrzl593DiDJU4ljOaY_c15tgPpGw8Xd-WnkPkiMhBn8Oz86wyuOhf9bsf96Lndvuu5ff_M2TgDz-1ddK763qXvel237131X86cn4bUvbi67PU6Xt_z3Y--e9ntv_wHEyE3kA?type=png)](https://mermaid.live/edit#pako:eNqtV1Fv4jgQ_itWXu6l7SZ0gQatTsqGdhddCxVQrXSqFJnEgLWJnbOd7bGl__3GTgA3JGyvWh7ajPPNZ898nrHz7MQ8Ic7AIWJI8Urg7JEh-AVhOHkYz9FzaeqfVIKyFcJxzAumIpqg-7_QoxOUNhoNH50jcEL1Q2zQNxo9rAZq8AQrUv45EIaCYEU5Q0MYb-BeCvJPQVi8AZ-RlAVmMUE3u8Gdw0v5L7wdXTeGE6eUWNGExmxa3YIKtY5YkS2IAOBnbbYt7S1hVwsbjmb3k9loPpqMG1YHRPl-bUCSc0lNThqzfYjlpjWWYxVvfqmi2uRamMkTI-LDgySiFsNkOryeNqyei4SI_fIn2voNq1lg9j1SHGBTEtPcxPgZxk7w1tAVve2wTDlWCGdmWkg1WVBFEhRkdWDF_D2Sm2zBU8De402mWcM1FjhWRNCfZt_WkjSfBuNZELYIrQRmcp-qubaAq1XptyXMriqbsmXXVjJD3SVUffhG1ToR-AmnDVCeE1EW52vmO2gmJ9JqQ49TW4IXODWVDCVWPQVLSCqyfE_rYU_SpkltOxkVhYLt_YudZAFrm6hS-XYSNMkLkbG9urdg_C5ZDdcXiFjv1rqutewb7HHaKbxMir2cBjWs7GO2vNztEpB3nKl1ukFVAciGgKTCqpA71pmx6p05mA6b-jIWyaErg9F6wOR2lz3dIHc7XNPN4fkoqxTOEUjkbsbSstN66Nrz6SicN7fsfe-_P3HkVXCGM2KDxmA3wARZlfJMzUNdP8YjvowoW2PoW7iUZ8wvEF-i0WGw2SsrGIXOiFNIHJHRE9S9zRSlKuq6rtb7FRB9QjD8PkpwhJ4ceb7vH_PCy3P95n3UHbfk9hu59dtz_93kKxV5wNCQjD_NeDNrXEL2ioTGPi4tU3LaoRAL3S5eqfmgx2wx0VTDG8r9B7TmFYkkdE-h70ZBOYBmZuDYoWAky1NuajiCNZDI7-oJrWE9F0F_-N03efdavHvN6QGEILkgjBRCRnCwmBzXwr-2QQhAyGvPN88yqqAhRrGgGeho4gnNcz2IU14926tndYBaD9huz8_58_7KPDDtLsa6I4_2lVpHVxfSgSlpSROY5ADeUVVY-46oHQIpeUzNBPqU3nlVlP_PyYZpz-227Mja5SuWLSsqb3wa8wUk0bK3Ie1rTxlsDNfCZvS2PD4Hpv8y9BW2ssY5Z05GRIZpAt8ppuE-OmpNdJ_U0IQscZGaI-0FoLhQfLZhsTNQoiBnTpHrzl593DiDJU4ljOaY_c15tgPpGw8Xd-WnkPkiMhBn8Oz86wyuOhf9bsf96Lndvuu5ff_M2TgDz-1ddK763qXvel237131X86cn4bUvbi67PU6Xt_z3Y--e9ntv_wHEyE3kA)

This ERD shows how the data appears in the dataset:

[![](https://mermaid.ink/img/pako:eNqtV99P2zAQ_lesvOyFbjCJSq2mSSHlRzRoUVq0F6TITdzWIrEz2xnqgP99ZydNTeIUhOgD5JzvPvvufJ-dJy_hKfHGHhETitcC5_cMwc8PgtnddIGeKlP_pBKUrRFOEl4yFdMU3f5C955f2Sic3HsdcEr1Q2LQFxo9qQda8BQrUv3ZEwaCYEU5QxMYd3CvBPlTEpZswSeUssQsIehiN7hzeKn-BdfhuTOcJKPEiiYwpmt1SyrUJmZlviQCgGfa7Fvae8KuFzYJ57ezebgIZ1PH6oCoaNYGJAWX1OTEme19LBe9sXSrePFmFdW20IWZPTIivt1JIloxzKLJeeRYPRcpEc3yZ9r6hNUsMXuIFQdYRBJamBjPYOwAbwtd09sOq4xjhXBupoVUkyVVJEV-3gbWzA-x3OZLngH2Fm9zzRpssMCJIoL-M_u2laRF5E_nftBTaCUwk02qFtoCrt5Kvy9hdlfZlD27ti4z9F1K1bffVG1SgR9x5oDygoiqOV8z34CYHEirDe2mtgIvcWY6GVqsfvJXkFRk-R6uhz1JX01a28lUUSjY3m_sJAvY2kR1la9nvqu8EBlrqnsNxmeV1XBdQsR6t7br2sq-wXbTTuFlWjblNKhJbXfZimq3S0DecKY22RbVDSAdAUmFVSl3rHNjtZXZjyYuXcYi3asyGL0HTGGr7GGB3O1wTbeA505WKZwjkMjdjJVlp3Wv2osoDBZuyW60__bAkVfDGc6JDZqC7YAJsq7KE5mHdv0Yj_kqpmyDQbdwVZ4p_4r4CoX7QbdXXjIKyogzSByR8SP0vc0UZyo-PT7W9X4FRD8QDH-MEhxBk-OT0WjU5YWXA_3mY9TfjyvukZNbvx2MPky-VvEJMDiS8dOMu1mTCtJUJDB2t7VMy2mHUiy1XLyq5p0es4uJIg13tPtfkOY1iSWop9B3I78aQHMz0HUoGcmLjJsejmENJB6d6gmtYT0XQV9Gp-_yHvZ4D93pAYQghSCMlELGcLCYHLfCP7dBCEDopD_fPM-pAkGME0FzqKOJJzDP7SAOeQ1tr6GlAC0NeH4eDPhTc2UeG7lLsFbksOnUNrq-kI5NS0uawiR78I6qxtp3RO3gS8kTaibQp_TOq6bUTs_P73WyYbWnUWTtcoVlz4qqG5_GXEJJdNn7kPa1pwo2gWuhEw1Tm-NzbPSXoSvYyhrnHXk5ETmmKXynGMG999SGaJ3U0JSscJmZI-0FoLhUfL5liTdWoiRHXlloZa8_brzxCmcSRvUVh4ub6tsn4WxF197LfwBRISI?type=png)](https://mermaid.live/edit#pako:eNqtV99P2zAQ_lesvOyFbjCJSq2mSSHlRzRoUVq0F6TITdzWIrEz2xnqgP99ZydNTeIUhOgD5JzvPvvufJ-dJy_hKfHGHhETitcC5_cMwc8PgtnddIGeKlP_pBKUrRFOEl4yFdMU3f5C955f2Sic3HsdcEr1Q2LQFxo9qQda8BQrUv3ZEwaCYEU5QxMYd3CvBPlTEpZswSeUssQsIehiN7hzeKn-BdfhuTOcJKPEiiYwpmt1SyrUJmZlviQCgGfa7Fvae8KuFzYJ57ezebgIZ1PH6oCoaNYGJAWX1OTEme19LBe9sXSrePFmFdW20IWZPTIivt1JIloxzKLJeeRYPRcpEc3yZ9r6hNUsMXuIFQdYRBJamBjPYOwAbwtd09sOq4xjhXBupoVUkyVVJEV-3gbWzA-x3OZLngH2Fm9zzRpssMCJIoL-M_u2laRF5E_nftBTaCUwk02qFtoCrt5Kvy9hdlfZlD27ti4z9F1K1bffVG1SgR9x5oDygoiqOV8z34CYHEirDe2mtgIvcWY6GVqsfvJXkFRk-R6uhz1JX01a28lUUSjY3m_sJAvY2kR1la9nvqu8EBlrqnsNxmeV1XBdQsR6t7br2sq-wXbTTuFlWjblNKhJbXfZimq3S0DecKY22RbVDSAdAUmFVSl3rHNjtZXZjyYuXcYi3asyGL0HTGGr7GGB3O1wTbeA505WKZwjkMjdjJVlp3Wv2osoDBZuyW60__bAkVfDGc6JDZqC7YAJsq7KE5mHdv0Yj_kqpmyDQbdwVZ4p_4r4CoX7QbdXXjIKyogzSByR8SP0vc0UZyo-PT7W9X4FRD8QDH-MEhxBk-OT0WjU5YWXA_3mY9TfjyvukZNbvx2MPky-VvEJMDiS8dOMu1mTCtJUJDB2t7VMy2mHUiy1XLyq5p0es4uJIg13tPtfkOY1iSWop9B3I78aQHMz0HUoGcmLjJsejmENJB6d6gmtYT0XQV9Gp-_yHvZ4D93pAYQghSCMlELGcLCYHLfCP7dBCEDopD_fPM-pAkGME0FzqKOJJzDP7SAOeQ1tr6GlAC0NeH4eDPhTc2UeG7lLsFbksOnUNrq-kI5NS0uawiR78I6qxtp3RO3gS8kTaibQp_TOq6bUTs_P73WyYbWnUWTtcoVlz4qqG5_GXEJJdNn7kPa1pwo2gWuhEw1Tm-NzbPSXoSvYyhrnHXk5ETmmKXynGMG999SGaJ3U0JSscJmZI-0FoLhUfL5liTdWoiRHXlloZa8_brzxCmcSRvUVh4ub6tsn4WxF197LfwBRISI)

In [25]:
# Verify 1:1 relationships between CLIENT, ACCOUNT, LOAN and DISPOSITION
assert dispositions['client_id'].is_unique, "Each client_id should appear exactly once in the DISPOSITION DataFrame."
assert dispositions['account_id'].is_unique == False, "An account_id should appear more than once in the DISPOSITION DataFrame."
assert loans['account_id'].is_unique, "Each account_id should appear exactly once in the LOAN DataFrame."

# Verify each district_id in ACCOUNT and CLIENT exists in DISTRICT
assert set(accounts['district_id']).issubset(set(districts['district_id'])), "All district_ids in ACCOUNT should exist in DISTRICT."
assert set(clients['district_id']).issubset(set(districts['district_id'])), "All district_ids in CLIENT should exist in DISTRICT."

# Verify each account_id in DISPOSITION, ORDER, TRANSACTION, and LOAN exists in ACCOUNT
assert set(dispositions['account_id']).issubset(set(accounts['account_id'])), "All account_ids in DISPOSITION should exist in ACCOUNT."
assert set(orders['account_id']).issubset(set(accounts['account_id'])), "All account_ids in ORDER should exist in ACCOUNT."
assert set(transactions['account_id']).issubset(set(accounts['account_id'])), "All account_ids in TRANSACTION should exist in ACCOUNT."
assert set(loans['account_id']).issubset(set(accounts['account_id'])), "All account_ids in LOAN should exist in ACCOUNT."

# Verify each client_id in DISPOSITION exists in CLIENT
assert set(dispositions['client_id']).issubset(set(clients['client_id'])), "All client_ids in DISPOSITION should exist in CLIENT."

# Verify each disp_id in CARD exists in DISPOSITION
assert set(cards['disp_id']).issubset(set(dispositions['disp_id'])), "All disp_ids in CARD should exist in DISPOSITION."

In [22]:
dispositions[dispositions["account_id"] == 3980]

,disp_id,client_id,account_id,type
4539,4799,4799,3980,OWNER
4540,4800,4800,3980,DISPONENT


## Relationships of Account

- Account:Loan 1:1

In [ ]:
assert loans['account_id'].nunique() == len(loans)

- Account:Transaction 1:n

- Account:Order 1:n

In [ ]:
orders_pivot = orders.pivot_table(index='account_id', 
                                 columns='k_symbol', 
                                 values='debited_amount',
                                 aggfunc='sum', 
                                 fill_value=0)

orders_pivot.columns = [f'k_symbol_debited_sum_{col}' for col in orders_pivot.columns]

# TODO: find something better than this
orders_pivot = orders_pivot.reset_index() # Use created index as account_id

orders_pivot.head()

,account_id,k_symbol_debited_sum_Household,k_symbol_debited_sum_Insurance_Payment,k_symbol_debited_sum_Leasing,k_symbol_debited_sum_Loan_Payment
0,1,2452.0,0.0,0.0,0.0
1,2,7266.0,0.0,0.0,3372.7
2,3,1135.0,3539.0,0.0,0.0
3,4,3363.0,0.0,0.0,0.0
4,5,2668.0,0.0,0.0,0.0


- Account:Disposition 1:n

- Account:District n:1

## Relationship of Client

- Client:Disposition 1:1

- Client:District n:1

## Relationship of Disposition

- Disposition:Card 1:1

## Merge non-transactional data

In [ ]:
def merge_non_transactional_data(clients, districts, dispositions, accounts, orders, loans, cards):
    merge = pd.merge(dispositions, accounts, on='account_id', how='left')

    merge = pd.merge(merge, clients, on='client_id', how='left')

    # TODO: suffixes do not work?
    merge = pd.merge(merge, cards, on='disp_id', how='left', suffixes=('', '_card'))

    merge = pd.merge(merge, loans, on='account_id', how='left', suffixes=('', '_loan'))

    merge = pd.merge(merge, orders, on='account_id', how='left', suffixes=('', '_order'))

    # TODO: ugly hack because merges before create two district_id columns
    merge = merge.drop(columns=['district_id_y'])
    merge.rename(columns={'district_id_x': 'district_id'}, inplace=True)
    merge = pd.merge(merge, districts, on='district_id', how='left', suffixes=('', '_district'))
    
    return merge



In [26]:
def merge_non_transactional_data_2(clients, districts, dispositions, accounts, orders, loans, cards):
    # Rename district_id for clarity
    clients = clients.rename(columns={'district_id': 'client_district_id'})
    accounts = accounts.rename(columns={'district_id': 'account_district_id'})
    
    # Prepare districts dataframe for merge with prefix for clients
    districts_client_prefixed = districts.add_prefix('client_')
    districts_account_prefixed = districts.add_prefix('account_')
    
    # Merge district information for clients and accounts with prefixed columns
    clients_with_districts = pd.merge(clients, districts_client_prefixed, left_on='client_district_id', right_on='client_district_id', how='left')
    accounts_with_districts = pd.merge(accounts, districts_account_prefixed, left_on='account_district_id', right_on='account_district_id', how='left')
    
    # Merge cards with dispositions
    dispositions_with_cards = pd.merge(dispositions, cards, on='disp_id', how='left')
    
    # Merge clients (with district info) with dispositions and cards
    clients_dispositions_cards = pd.merge(dispositions_with_cards, clients_with_districts, on='client_id', how='left')
    
    # Merge the above with accounts (with district info) on account_id
    accounts_clients_cards = pd.merge(accounts_with_districts, clients_dispositions_cards, on='account_id', how='left')
    
    # Merge loans with the comprehensive dataframe on account_id
    final_df = pd.merge(accounts_clients_cards, loans, on='account_id', how='left')
    
    return final_df


In [ ]:
non_transactional_df = merge_non_transactional_data(clients, districts, dispositions, accounts, orders_pivot, loans, cards)
non_transactional_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   disp_id                                 4500 non-null   int64         
 1   client_id                               4500 non-null   int64         
 2   account_id                              4500 non-null   int64         
 3   type                                    4500 non-null   category      
 4   district_id                             4500 non-null   int64         
 5   account_frequency                       4500 non-null   category      
 6   account_created                         4500 non-null   datetime64[ns]
 7   client_sex                              4500 non-null   object        
 8   birth_date                              4500 non-null   datetime64[ns]
 9   age                                     4500 non-nul

In [ ]:
before = len(dispositions)
dispositions = dispositions[dispositions['type'] == "OWNER"]
after = len(dispositions)
print(f"Dropped {before - after} dispositions that are not of type 'OWNER'.")

Dropped 869 dispositions that are not of type 'OWNER'.


In [27]:
non_transactional_df_2 = merge_non_transactional_data_2(clients, districts, dispositions, accounts, orders_pivot, loans, cards)
non_transactional_df_2.info()

NameError: name 'orders_pivot' is not defined

In [ ]:
# Convert DataFrame columns to sets for easy set operations
columns_df1 = set(non_transactional_df.columns)
columns_df2 = set(non_transactional_df_2.columns)

# Find common columns
common_columns = columns_df1.intersection(columns_df2)
print("Common columns:", common_columns)

# Find columns unique to non_transactional_df
unique_to_df1 = columns_df1.difference(columns_df2)
print("Columns unique to non_transactional_df:", unique_to_df1)

# Find columns unique to non_transactional_df_2
unique_to_df2 = columns_df2.difference(columns_df1)
print("Columns unique to non_transactional_df_2:", unique_to_df2)


Common columns: {'client_sex', 'loan_monthly_payments', 'loan_status', 'card_type', 'loan_granted_date', 'loan_id', 'disp_id', 'loan_amount', 'birth_date', 'age', 'account_id', 'card_issued', 'client_id', 'loan_duration', 'card_id', 'account_frequency', 'account_created'}
Columns unique to non_transactional_df: {'region', 'k_symbol_debited_sum_Leasing', 'district_id', 'k_symbol_debited_sum_Household', 'k_symbol_debited_sum_Loan_Payment', 'small_municipalities', 'huge_municipalities', 'average_salary', 'k_symbol_debited_sum_Insurance_Payment', 'medium_municipalities', 'unemployment_rate_1996', 'inhabitants', 'ratio_urban_inhabitants', 'crimes_committed_1996', 'crimes_committed_1995', 'district_name', 'unemployment_rate_1995', 'cities', 'entrepreneurs_per_1000_inhabitants', 'large_municipalities'}
Columns unique to non_transactional_df_2: {'account_average_salary', 'account_district_id', 'client_cities', 'client_inhabitants', 'account_unemployment_rate_1995', 'client_crimes_committed_199

## Transactional Data

# Model Construction

# Feature Engineering

# Model Engineering

# Model Comparison & Selection

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7d865ccc-7b5c-4f8d-b4e6-4e008791345d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>